In [1]:
import os.path
import numpy as np
import pandas as pd

from keras.models import Sequential
from keras.layers import LSTM, Dense
from keras.callbacks import TensorBoard

from sklearn.model_selection import train_test_split
from keras.utils import to_categorical

In [2]:
pose_classes = ['hands_up', 'neck_down', 'neck_side']

# one hot encoding
def onehot(y):
    label = []
    for i in y:
        label.append([int(i)])
    y_result = to_categorical(label).astype(int)
    
    return y_result

In [3]:
# load data
train = np.loadtxt('../data/csvs/train_data.csv', delimiter=",")
test = np.loadtxt('../data/csvs/test_data.csv', delimiter=",")

In [4]:
X_train = train[:, :-1]
y_train_split = train[:, -1]
X_test = test[:, :-1]
y_test_split = test[:, -1]

y_train = onehot(y_train_split)
y_test = onehot(y_test_split)

In [29]:
X_train = np.array([X_train])
y_train = np.array([y_train])

In [20]:
seq = []
for i in range(3):
    temp = []
    for j in range(139):
        window = []
        if y_train_split[j] == i:
            window.append(X_train[j])
        temp.append(window)
    seq.append(temp)


In [21]:
seq = np.array(seq)
seq.shape

C:\Users\miming\AppData\Local\Temp\ipykernel_18236\3748410438.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  seq = np.array(seq)


(3, 139)

In [5]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)


(139, 22)
(15, 22)
(139, 3)
(15, 3)


In [23]:
# logging
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [24]:
# model learning
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(139,22)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(3, activation='softmax'))

In [25]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [30]:
model.fit(X_train, y_train, epochs=100, callbacks=[tb_callback])

Epoch 1/100


ValueError: in user code:

    File "c:\Users\miming\anaconda3\envs\yolo-pose\lib\site-packages\keras\engine\training.py", line 1249, in train_function  *
        return step_function(self, iterator)
    File "c:\Users\miming\anaconda3\envs\yolo-pose\lib\site-packages\keras\engine\training.py", line 1233, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\Users\miming\anaconda3\envs\yolo-pose\lib\site-packages\keras\engine\training.py", line 1222, in run_step  **
        outputs = model.train_step(data)
    File "c:\Users\miming\anaconda3\envs\yolo-pose\lib\site-packages\keras\engine\training.py", line 1023, in train_step
        y_pred = self(x, training=True)
    File "c:\Users\miming\anaconda3\envs\yolo-pose\lib\site-packages\keras\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "c:\Users\miming\anaconda3\envs\yolo-pose\lib\site-packages\keras\engine\input_spec.